In [1]:
from bs4 import BeautifulSoup
import requests

In [4]:
import bs4

print(bs4.__version__)
print(requests.__version__)

4.12.3
2.31.0


# Empezamos el scraping

## 1) Obtener el HTML

In [5]:
URL_BASE = 'https://scrapepark.org/spanish/'
pedido_obtenido = requests.get(URL_BASE)
html_obtenido = pedido_obtenido.text

## 2) Parsear el HTML

In [6]:
soup = BeautifulSoup(html_obtenido,"html.parser")

## El método find() nos permite quedarnos con la información asociada a una etiqueta de HTML

In [ ]:
primer_h2 = soup.find('h2')
print(primer_h2)
print(primer_h2.text)

## El método find_all() busca todos los elementos de la página con esa etiqueta y devuelve una "lista" que los contiene (en realidad devuelve un objeto de la clase bs4.element.ResultSet).

In [ ]:
h2_todos=soup.find_all('h2')
print(h2_todos)

## Si usamos el parametro limit=1, emulamos al método find

In [ ]:
h2_uno_solo = soup.find_all('h2',limit=1)
print(h2_uno_solo)

## Podemos iterar sobre el objeto

In [ ]:
h2=soup.find_all('h2')
for i in h2:
    print(i.text)

## get_text() para más funcionalidades

In [ ]:
for seccion in h2:
    print(seccion.get_text(strip=True))

## Utilizando atributos de las etiquetas

## Clase

In [8]:
divs = soup.find_all('div', class_ = 'heading-container heading-center')
for div in divs:
    print(div)
    print(' ')

<div class="heading-container heading-center" id="about">
<h2>¿Por qué comprar con nosotros?</h2>
</div>
 
<div class="heading-container heading-center" id="products">
<h2>Nuestros productos</h2>
</div>
 
<div class="heading-container heading-center">
<h3>Suscríbete para obtener descuentos y ofertas</h3>
</div>
 
<div class="heading-container heading-center">
<h2>Testimonios de clientes</h2>
</div>
 


# Todas las etiquetas que tengan el atributo "src"

In [25]:
src_todos = soup.find_all('img')
for elemento in src_todos:
    print(elemento['alt'])
    '''if elemento['src'].endswith('.jpg'):
        print(elemento)'''

ScrapePark.org Logo
Parque de patinaje
Variedad de patinetas en nuestra tienda
Patineta
Patineta
Patineta
Patineta
Patineta
Patineta
Patineta
Patineta
Patineta
Patineta
Patineta
Patineta
Cliente
Cliente
Cliente
Logo de ScrapePark.org
Logo de freeCodeCamp


# Bajar todas las imagenes

In [ ]:
for i,imagen in enumerate(src_todos):
    if imagen['src'].endswith('png'):
        print(imagen['src'])
        r=requests.get(f'https://scrapepark.org/spanish/{imagen['src']}')
        with open(f'imagen_{i}.png','wb') as f:
            f.write(r.content)            

# Información de tablas

In [27]:
URL_BASE = 'https://scrapepark.org/spanish'
URL_TABLA = soup.find_all('iframe')[0]['src']

request_tabla = requests.get(f'{URL_BASE}/{URL_TABLA}')

html_tabla = request_tabla.text
soup_tabla = BeautifulSoup(html_tabla,'html.parser')
soup_tabla.find('table')

productos_faltantes = soup_tabla.find_all(['th','td'], attrs={'style':'color: red;'})
productos_fal = [talle.text for talle in productos_faltantes ]

print(productos_fal)

['Longboard', '$80', '$85', '$90', '$62', '$150']


In [ ]:
divs = soup.find_all('div', class_='detail-box')
productos = []
precios = []

for div in divs:
    if div.h6 is not None and 'Patineta' in div.h5.text:
        producto = div.h5.get_text(strip=True)
        precio = div.h6.get_text(strip=True).replace('$','')
        print(f'producto: {producto:<16} | precio: {precio}')
        productos.append(producto)
        precios.append(precio)
print(*productos)
print(*precios)        

# Cambios que dependen de la URL

In [31]:
URL_BASE = 'https://scrapepark.org/spanish/contact'

for i in range(1,3):
    URL_FINAL = f'{URL_BASE}{i}'
    print(URL_FINAL)
    r = requests.get(URL_FINAL)
    soup = BeautifulSoup(r.text,'html.parser')
    print(soup.h5.text) 

https://scrapepark.org/spanish/contact1
Texto que cambia entre páginas en contacto 1 :)
https://scrapepark.org/spanish/contact2
Texto que cambia entre páginas en contacto 2 :)


# Datos que no sabemos en que parte de la página se encuentran

In [32]:
# Expresiones regulares
import re

# 1. Obtener el HTML
URL_BASE = 'https://scrapepark.org/spanish'
pedido_obtenido = requests.get(URL_BASE)
html_obtenido = pedido_obtenido.text

# 2.'Parsear' ese HTML
soup = BeautifulSoup(html_obtenido,'html.parser')
telefonos = soup.find_all(string=re.compile('\d+-\d+-\d+'))
print(telefonos)

<>:11: SyntaxWarning: invalid escape sequence '\d'
<>:11: SyntaxWarning: invalid escape sequence '\d'
C:\Users\jamir\AppData\Local\Temp\ipykernel_14364\229612336.py:11: SyntaxWarning: invalid escape sequence '\d'
  telefonos = soup.find_all(string=re.compile('\d+-\d+-\d+'))


[': 4-444-4444']
